# One Supporting Fact Model

In [1]:
%matplotlib inline
import importlib, utils2; 
importlib.reload(utils2)
from utils2 import *
import sys

Using TensorFlow backend.


In [2]:
np.set_printoptions(4)

In [3]:
cfg = K.tf.ConfigProto(gpu_options = {'allow_growth': True})
K.set_session(K.tf.Session(config = cfg))

In [4]:
def tokenize(sent):
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip() ]

In [5]:
def parse_stories(lines):
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid,line = line.split(" ",1)
        if int(nid) == 1:story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            substory = [[str(i)+":"]+x for i,x in enumerate(story) if x]
            data.append((substory, q, a))
            story.append('')
        else: story.append(tokenize(line))
    return data

In [6]:
path = get_file('babi-tasks-v1-2.tar.gz', origin = 'https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')

In [7]:
tar = tarfile.open(path)

In [8]:
challenges = {
    # QA1 with 10,000 samples
    'single_supporting_fact_10k': 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt',
    # QA2 with 10,000 samples
    'two_supporting_facts_10k': 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt',
    'two_supporting_facts_1k': 'tasks_1-20_v1-2/en/qa2_two-supporting-facts_{}.txt',
}

In [9]:
challenge_type = 'single_supporting_fact_10k'

challenge = challenges[challenge_type]

In [10]:
def get_stories(file):
    data = parse_stories(file.readlines())
    return [(story, question, answer) for story, question, answer in data]

In [11]:
train_stories = get_stories(tar.extractfile(challenge.format('train')))
test_stories = get_stories(tar.extractfile(challenge.format('test')))

C:\Users\kpranoy\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [12]:
stories = train_stories + test_stories

In [13]:
story_maxlen =    max((len(x) for s,_,_ in stories for x in s))
story_maxsents = max((len(x) for x, _, _ in stories))
query_maxlen = max(len(x) for _, x, _ in stories)

In [14]:
import time
def create_vocab(stories):
    vocab = set()
    for i,story in enumerate(stories):
        sys.stdout.write("\r Running story number: " + str(i))
        
        #Getting vocab from stories
        for text in story[0]:
            [vocab.add(word) for word in text ]
        sys.stdout.flush()
        time.sleep(0.001)
        #getting vocab from questions
        [vocab.add(word) for word in story[1] ]
        
        #Getting vocab from Answer
        vocab.add(story[2])
    return vocab

In [15]:
vocab = sorted(create_vocab(stories))
vocab.insert(0, '<PAD>')
vocab_size = len(vocab)

 Running story number: 9700

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 Running story number: 10999

In [16]:
story_maxsents, vocab_size, story_maxlen, query_maxlen, len(train_stories), len(test_stories)

(10, 32, 8, 4, 10000, 1000)

In [17]:
word_idx = dict((c, i) for i, c in enumerate(vocab))

In [18]:
def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    X = []; Xq = []; Y = []
    for story, query, answer in data:
        x = [[word_idx[w] for w in s] for s in story]
        xq = [word_idx[w] for w in query]
        y = [word_idx[answer]]
        X.append(x); Xq.append(xq); Y.append(y)
    return ([pad_sequences(x, maxlen=story_maxlen) for x in X],
            pad_sequences(Xq, maxlen=query_maxlen), np.array(Y))

In [19]:
inputs_train, queries_train, answers_train = vectorize_stories(train_stories, 
     word_idx, story_maxlen, query_maxlen)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories, 
     word_idx, story_maxlen, query_maxlen)

In [20]:
def stack_inputs(inputs):
    for i,it in enumerate(inputs):
        inputs[i] = np.concatenate([it, 
                           np.zeros((story_maxsents-it.shape[0],story_maxlen), 'int')])
    return np.stack(inputs)
inputs_train = stack_inputs(inputs_train)
inputs_test = stack_inputs(inputs_test)

In [21]:
inputs_train.shape, inputs_test.shape

((10000, 10, 8), (1000, 10, 8))

In [22]:
inps = [inputs_train, queries_train]
val_inps = [inputs_test, queries_test]

In [23]:
emb_dim = 20
parms = {'verbose': 2}

In [25]:
def emb_sent_bow(inp):
    emb = TimeDistributed(Embedding(vocab_size, emb_dim))(inp)
    return Lambda(lambda x: K.sum(x, 2))(emb)

In [26]:
inp_story = Input((story_maxsents, story_maxlen))
emb_story = emb_sent_bow(inp_story)
inp_story.get_shape(), emb_story.get_shape()

(TensorShape([Dimension(None), Dimension(10), Dimension(8)]),
 TensorShape([Dimension(None), Dimension(10), Dimension(20)]))

In [27]:
inp_q = Input((query_maxlen,))

In [32]:
emb_q = Embedding(vocab_size, emb_dim)(inp_q)
emb_q = Lambda(lambda x: K.sum(x, 1))(emb_q)
emb_q = Reshape((1, emb_dim))(emb_q)
inp_q.get_shape(), emb_q.get_shape()

(TensorShape([Dimension(None), Dimension(4)]),
 TensorShape([Dimension(None), Dimension(1), Dimension(20)]))

In [33]:
x = merge([emb_story, emb_q], mode='dot', dot_axes=2)
x = Reshape((story_maxsents,))(x)
x = Activation('softmax')(x)
match = Reshape((story_maxsents,1))(x)
match.shape

C:\Users\kpranoy\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  """Entry point for launching an IPython kernel.
C:\Users\kpranoy\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


TensorShape([Dimension(None), Dimension(10), Dimension(1)])

In [34]:
emb_c = emb_sent_bow(inp_story)
x = merge([match,emb_c], mode='dot', dot_axes=1)
response = Reshape((emb_dim,),)(x)
res = Dense(vocab_size, activation= 'softmax')(response)

C:\Users\kpranoy\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
C:\Users\kpranoy\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [35]:
answer = Model([inp_story, inp_q], res)
answer.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [36]:
K.set_value(answer.optimizer.lr, 1e-2)
hist = answer.fit(inps, answers_train, **parms, nb_epoch=4, batch_size=32,
                 validation_data=(val_inps, answers_test))

C:\Users\kpranoy\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 10000 samples, validate on 1000 samples
Epoch 1/4
0s - loss: 0.3701 - acc: 0.8794 - val_loss: 2.2982e-04 - val_acc: 1.0000
Epoch 2/4
0s - loss: 0.0055 - acc: 0.9979 - val_loss: 0.1486 - val_acc: 0.9750
Epoch 3/4
0s - loss: 0.0062 - acc: 0.9986 - val_loss: 5.0378e-06 - val_acc: 1.0000
Epoch 4/4
0s - loss: 0.0140 - acc: 0.9976 - val_loss: 0.0265 - val_acc: 0.9960


## Testing our Model

In [55]:
q_num = 117
f = Model([inp_story, inp_q],match)

In [56]:
train_stories[q_num]

([['0:', 'Mary', 'went', 'back', 'to', 'the', 'hallway', '.'],
  ['1:', 'Daniel', 'went', 'back', 'to', 'the', 'bedroom', '.'],
  ['3:', 'Sandra', 'moved', 'to', 'the', 'bathroom', '.'],
  ['4:', 'Sandra', 'journeyed', 'to', 'the', 'hallway', '.'],
  ['6:', 'Mary', 'went', 'back', 'to', 'the', 'bedroom', '.'],
  ['7:', 'Mary', 'went', 'back', 'to', 'the', 'garden', '.']],
 ['Where', 'is', 'Mary', '?'],
 'garden')

Trying to Predict which sentence has the highest weight on a pretrained model for better understanding

In [59]:
np.squeeze(f.predict([inputs_train[q_num:q_num+1],queries_train[q_num:q_num+1
                                             ]]))[:(len(train_stories[q_num][0]) + 1)]

array([  2.4759e-07,   7.7565e-13,   9.0063e-11,   4.4669e-10,
         9.2971e-02,   9.0703e-01,   1.2456e-18], dtype=float32)

Now applying it to our model

In [63]:
answers_train[q_num:q_num+10,0]

array([21, 27, 27, 22, 25, 20, 20, 20, 25, 19])

In [66]:
np.argmax(answer.predict([inputs_train[q_num:q_num+10], queries_train[q_num:q_num+10]]),1)

array([21, 27, 27, 22, 25, 20, 20, 20, 25, 19], dtype=int64)

# Two Supporting Facts

In [67]:
test_stories[534]

([['0:', 'Mary', 'moved', 'to', 'the', 'office', '.'],
  ['1:', 'John', 'moved', 'to', 'the', 'garden', '.'],
  ['3:', 'Sandra', 'moved', 'to', 'the', 'bedroom', '.'],
  ['4:', 'Sandra', 'went', 'back', 'to', 'the', 'office', '.'],
  ['6:', 'John', 'went', 'to', 'the', 'bedroom', '.'],
  ['7:', 'John', 'journeyed', 'to', 'the', 'garden', '.'],
  ['9:', 'Daniel', 'went', 'back', 'to', 'the', 'hallway', '.'],
  ['10:', 'John', 'journeyed', 'to', 'the', 'bedroom', '.'],
  ['12:', 'Daniel', 'journeyed', 'to', 'the', 'bathroom', '.'],
  ['13:', 'John', 'travelled', 'to', 'the', 'garden', '.']],
 ['Where', 'is', 'Daniel', '?'],
 'bathroom')